In [24]:
#r "nuget: FsToolkit.ErrorHandling, 4.12.0"

Installed Packages FsToolkit.ErrorHandling, 4.12.0

In [25]:
open System
open FsToolkit.ErrorHandling

Let's see an example of the result data type which can have either of the two values (this is also present in Scala as Either, however in Scala, the left part is an error, whereas the Right part is Success). In F#, the left part is Ok, whereas the right part is Error.

In [26]:
let divide nr dr = 
  if (dr = 0) then Error("Cannot divide by 0.")
  else Ok(nr / dr)

The type of the divide function is `int -> int -> Result<int, string>`.

In [27]:
let numerator = 4
let denominator = 2

let okResult = divide numerator denominator
let errorResult = divide numerator 0

Below is how we pattern match on a result value.

In [28]:
match okResult with
  | Ok(result) -> printfn $"The result of dividing {numerator} with {denominator} is: {result}"
  | Error(e) -> printfn $"Error: {e}"

The result of dividing 4 with 2 is: 2


In [29]:
match errorResult with
  | Ok(result) -> printfn $"The result of dividing {numerator} with {denominator} is: {result}"
  | Error(e) -> printfn $"Error: {e}"

Error: Cannot divide by 0.


Careful use of Result values also helps us to reduce / eliminate NPEs in the code. **Not just that, it also helps us communicate the cause of the problems.**

### Again, you might say that this pattern matching is not a big deal. We could've used the trusted old if-else statement?

First, let's see how to get the value of a result safely.

In [30]:
let okVal = Result.defaultValue 0 ok
printfn $"okVal: {okVal}"

okVal: 2


Or in an idiomatic F# way -

In [31]:
let iokVal = ok |> Result.defaultValue 0
printfn $"iokVal: {iokVal}"

iokVal: 2


Now, let's see how we can use multiple result values together. This is something which is not possible in C#. But `FsToolkit.ErrorHandling` makes it a walk in the park.

In [32]:
let tryParseInt (str: string) =
    match Int32.TryParse str with
    | (true, i) -> Ok i
    | _ -> Error $"Could not parse string {str} to int."

let printResult result =
    match result with
    | Ok i -> printfn $"The result is: {i}"
    | Error e -> printfn $"Error: {e}"

let printResultWithErrors (result: Result<'a list, 'b list>) =
    match result with
    | Ok is -> printfn $"The result is: {is}"
    | Error es -> printfn $"Errors: {es}"

In [33]:
let addAllOks = result {
    let! x = tryParseInt "7"
    let! y = tryParseInt "5"
    let! z = tryParseInt "2"
    return x + y + z
}

printResult addAllOks

The result is: 14


In [34]:
let addOksWithError = result {
    let! x = tryParseInt "7"
    let! y = tryParseInt "sad"
    let! z = tryParseInt "2"
    return x + y + z
}

printResult addOksWithError

Error: Could not parse string sad to int.


If you have a list of string values, and you use the above function, you get a list of result values.

In [35]:
let listOfResults =
    ["1"; "2"; "3"]
    |> List.map tryParseInt

printfn $"listOfResults (which is of type list<Result<int, string>>):"
listOfResults |> List.iter printResult

listOfResults (which is of type list<Result<int, string>>):
The result is: 1
The result is: 2
The result is: 3


But what if you want a Result<list<int>, string>? Simple, use `sequenceResultM` method.

In [36]:
let resultList = 
    ["1"; "2"; "3"]
    |> List.map tryParseInt
    |> List.sequenceResultM

printfn $"resultList (which is of type Result<list<int>, string>):"
printResult resultList

resultList (which is of type Result<list<int>, string>):
The result is: [1; 2; 3]


But what if one of the values is an error?

In [41]:
let resultList = 
    ["1"; "foo"; "3"; "bar"]
    |> List.map tryParseInt
    |> List.sequenceResultM

printfn $"resultList (which is of type Result<list<int>, string>):"
printResult resultList

resultList (which is of type Result<list<int>, string>):
Error: Could not parse string foo to int.


No, but we don't want that. We want to see all the errors. That's where `sequenceResultA` method comes into picture.
You may have guessed that the `M` stands for `Monad` whereas the `A` stands for `Applicative`.

In [38]:
let resultListWithAllErrors = 
    ["1"; "foo"; "3"; "bar"]
    |> List.map tryParseInt
    |> List.sequenceResultA

printfn $"resultListWithAllErrors (which is of type Result<list<int>, list<string>>):"
printResultWithErrors resultListWithAllErrors

resultListWithAllErrors (which is of type Result<list<int>, list<string>>):
Errors: [Could not parse string foo to int.; Could not parse string bar to int.]


Another compelling example of how to compose result values is related to validation. It is taken from the [Results](https://learn.microsoft.com/en-us/dotnet/fsharp/language-reference/results) page, but improved with the use of `FsToolkit.ErrorHandling`.

Notice that for the 3rd request, even though there were 2 errors, only the first one was reported. This is an example of a short circuiting behavior. So let's see how to get all the errors at once using `FsToolkit's Validation` data type.

In [40]:
let printValidated (validated: Validation<'a, 'b>) =
    match validated with
    | Ok i -> printfn $"The result is: {i}"
    | Error es -> printfn $"Errors: {es}"

let addResult1 = validation {
    let! x = tryParseInt "1"
    and! y = tryParseInt "str1"
    return x + y
}
printValidated addResult1

let addResult2 = validation {
    let! x = tryParseInt "1"
    and! y = tryParseInt "str2"
    return x + y
}
printValidated addResult2

let combinedResult = 
    validation {
        let! x = addResult1
        and! y = addResult2
        return x + y
    }
printValidated combinedResult

Errors: [Could not parse string str1 to int.]
Errors: [Could not parse string str2 to int.]
Errors: [Could not parse string str1 to int.; Could not parse string str2 to int.]
